### Добрый день!

Моё исследование сосредоточено на предсказании биологического ответа молекул на основе их химического состава с использованием методов машинного обучения. Я использовал логистическую регрессию и случайный лес, обучив их на молекулярных дескрипторах.

В процессе исследования я провёл анализ данных, разделил их на обучающий и тестовый наборы, обучил базовые модели и оценил их производительность. Затем я оптимизировал гиперпараметры, используя GridSearchCV, RandomizedSearchCV, Hyperopt и Optuna.

Эта работа стремится внести вклад в биоинформатику, предоставляя методологию и результаты, способные улучшить предсказательную способность моделей в биологических исследованиях.

In [33]:
import pandas as pd

# Загружаем данные
data = pd.read_csv('data/train_sem.csv')

# Разделяем данные на признаки (X) и целевую переменную (y)
X = data.drop('Activity', axis=1)
y = data['Activity']


In [34]:
from sklearn.model_selection import train_test_split, cross_val_score

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
# Обучаем логистическую регрессию и случайный лес.

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Логистическая регрессия
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train, y_train)

# Случайный лес
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier(random_state=42)

In [36]:
# Оцениваем модели на тестовом наборе

from sklearn.metrics import f1_score

# Предсказания
logreg_preds = logreg_model.predict(X_test)
rf_preds = rf_model.predict(X_test)

# Оценка производительности с использованием кросс-валидации
logreg_cv_score = cross_val_score(logreg_model, X_train, y_train, cv=5, scoring='f1').mean()
rf_cv_score = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='f1').mean()

print(f'F1-score для логистической регрессии: {logreg_cv_score}')
print(f'F1-score для случайного леса: {rf_cv_score}')


c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

F1-score для логистической регрессии: 0.7722178600966617
F1-score для случайного леса: 0.8056060934090883


Выбор конкретных методов оптимизации и моделей зависит от нескольких факторов, включая размер данных, характеристики данных, требования к модели, вычислительные ресурсы и другие особенности задачи. В данном случае мной были выбраны следующие методы и модели:

Логистическая регрессия:

Метод оптимизации: GridSearchCV и Hyperopt.
GridSearchCV прост в использовании и хорош для перебора значений в заданном диапазоне.
Hyperopt предоставляет гибкость в определении пространства поиска параметров и может эффективно оптимизировать целевую функцию.

Случайный лес:

Метод оптимизации: RandomizedSearchCV и Optuna.
RandomizedSearchCV полезен, когда пространство поиска параметров велико, и случайный выбор комбинаций параметров может быть более эффективным.
Optuna предоставляет более интеллектуальный и гибкий метод оптимизации, основанный на технике Sequential Model-Based Optimization (SMBO).

In [37]:
# Подбираем гиперпараметры
# Используем GridSearchCV, RandomizedSearchCV, Hyperopt и Optuna для каждой модели.

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from hyperopt import fmin, tpe, hp, Trials, space_eval
from optuna import create_study, logging

# GridSearchCV для логистической регрессии
param_grid_logreg = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search_logreg = GridSearchCV(LogisticRegression(random_state=42), param_grid_logreg, cv=5)
grid_search_logreg.fit(X_train, y_train)

# RandomizedSearchCV для случайного леса
param_dist_rf = {'n_estimators': [10, 50, 100, 200, 500],
                 'max_depth': [None, 10, 20, 30, 40, 50],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4]}
random_search_rf = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_dist_rf, n_iter=10, cv=5, random_state=42)
random_search_rf.fit(X_train, y_train)

# Hyperopt для логистической регрессии
space_logreg = {'C': hp.loguniform('C', -5, 3)}
trials_logreg = Trials()
best_logreg = fmin(fn=lambda params: -f1_score(y_train, LogisticRegression(C=params['C'], random_state=42).fit(X_train, y_train).predict(X_train)),
                   space=space_logreg,
                   algo=tpe.suggest,
                   max_evals=10,
                   trials=trials_logreg)

# Optuna для случайного леса
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 500)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_train)
    return -f1_score(y_train, preds)

study_rf = create_study(direction='minimize')
study_rf.optimize(objective, n_trials=10)


c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

 10%|█         | 1/10 [00:01<00:09,  1.07s/trial, best loss: -0.8509335782063054]

c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 30%|███       | 3/10 [00:03<00:08,  1.17s/trial, best loss: -0.9128268991282689]

c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



...

 90%|█████████ | 9/10 [00:09<00:01,  1.09s/trial, best loss: -0.9128268991282689]

c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



100%|██████████| 10/10 [00:11<00:00,  1.10s/trial, best loss: -0.9128268991282689]

c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

[I 2023-10-18 01:16:03,705] A new study created in memory with name: no-name-640fe222-6e6a-4394-876f-a1ec31d6317d


[I 2023-10-18 01:16:04,864] Trial 0 finished with value: -0.6844273551222192 and parameters: {'n_estimators': 190, 'max_depth': 9, 'min_samples_split': 0.30455607701164, 'min_samples_leaf': 0.1994367021082467}. Best is trial 0 with value: -0.6844273551222192.
[I 2023-10-18 01:16:05,200] Trial 1 finished with value: -0.6959356661595305 and parameters: {'n_estimators': 44, 'max_depth': 10, 'min_samples_split': 0.2500720762693943, 'min_samples_leaf': 0.31601842577563977}. Best is trial 1 with value: -0.6959356661595305.
[I 2023-10-18 01:16:06,805] Trial 2 finished with value: -0.6959356661595305 and parameters: {'n_estimators': 418, 'max_depth': 5, 'min_samples_split': 0.6762009064472112, 'min_samples_leaf': 0.11710680853579358}. Best is trial 1 with value: -0.6959356661595305.
[I 2023-10-18 01:16:08,050] Trial 3 finished with value: -0.6959356661595305 and parameters: {'n_estimators': 300, 'max_depth': 16, 'min_samples_split': 0.5520927169249237, 'min_samples_leaf': 0.37787391355767364}.

In [43]:
# Получение лучших параметров для GridSearchCV (логистическая регрессия)
best_params_gridsearchcv_logreg = grid_search_logreg.best_params_
logreg_model_best = LogisticRegression(**best_params_gridsearchcv_logreg, random_state=42)
logreg_model_best.fit(X_train, y_train)
logreg_train_preds = logreg_model_best.predict(X_train)
logreg_test_preds = logreg_model_best.predict(X_test)
logreg_train_f1 = f1_score(y_train, logreg_train_preds)
logreg_test_f1 = f1_score(y_test, logreg_test_preds)
print(f'Лучшие параметры для логистической регрессии (GridSearchCV): {best_params_gridsearchcv_logreg}')
print(f'Train F1-score для логистической регрессии (GridSearchCV): {round(logreg_train_f1, 2)}')
print(f'Test F1-score для логистической регрессии (GridSearchCV): {round(logreg_test_f1, 2)}')

# Получение лучших параметров для RandomizedSearchCV (случайный лес)
best_params_randomsearchcv_rf = random_search_rf.best_params_
rf_model_best = RandomForestClassifier(**best_params_randomsearchcv_rf, random_state=42)
rf_model_best.fit(X_train, y_train)
rf_train_preds = rf_model_best.predict(X_train)
rf_test_preds = rf_model_best.predict(X_test)
rf_train_f1 = f1_score(y_train, rf_train_preds)
rf_test_f1 = f1_score(y_test, rf_test_preds)
print(f'Лучшие параметры для случайного леса (RandomizedSearchCV): {best_params_randomsearchcv_rf}')
print(f'Train F1-score для случайного леса (RandomizedSearchCV): {round(rf_train_f1, 2)}')
print(f'Test F1-score для случайного леса (RandomizedSearchCV): {round(rf_test_f1, 2)}')

# Получение лучших параметров для Hyperopt (логистическая регрессия)
best_params_hyperopt_logreg = space_eval(space_logreg, best_logreg)
logreg_model_best_hyperopt = LogisticRegression(**best_params_hyperopt_logreg, random_state=42)
logreg_model_best_hyperopt.fit(X_train, y_train)
logreg_train_preds_hyperopt = logreg_model_best_hyperopt.predict(X_train)
logreg_test_preds_hyperopt = logreg_model_best_hyperopt.predict(X_test)
logreg_train_f1_hyperopt = f1_score(y_train, logreg_train_preds_hyperopt)
logreg_test_f1_hyperopt = f1_score(y_test, logreg_test_preds_hyperopt)
print(f'Лучшие параметры для логистической регрессии (Hyperopt): {best_params_hyperopt_logreg}')
print(f'Train F1-score для логистической регрессии (Hyperopt): {round(logreg_train_f1_hyperopt, 2)}')
print(f'Test F1-score для логистической регрессии (Hyperopt): {round(logreg_test_f1_hyperopt, 2)}')

# Получение лучших параметров для Optuna (случайный лес)
best_params_optuna_rf = study_rf.best_params
rf_model_best_optuna = RandomForestClassifier(**best_params_optuna_rf, random_state=42)
rf_model_best_optuna.fit(X_train, y_train)
rf_train_preds_optuna = rf_model_best_optuna.predict(X_train)
rf_test_preds_optuna = rf_model_best_optuna.predict(X_test)
rf_train_f1_optuna = f1_score(y_train, rf_train_preds_optuna)
rf_test_f1_optuna = f1_score(y_test, rf_test_preds_optuna)
print(f'Лучшие параметры для случайного леса (Optuna): {best_params_optuna_rf}')
print(f'Train F1-score для случайного леса (Optuna): {round(rf_train_f1_optuna, 2)}')
print(f'Test F1-score для случайного леса (Optuna): {round(rf_test_f1_optuna, 2)}')

c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Лучшие параметры для логистической регрессии (GridSearchCV): {'C': 0.1}
Train F1-score для логистической регрессии (GridSearchCV): 0.85
Test F1-score для логистической регрессии (GridSearchCV): 0.79
Лучшие параметры для случайного леса (RandomizedSearchCV): {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 50}
Train F1-score для случайного леса (RandomizedSearchCV): 0.99
Test F1-score для случайного леса (RandomizedSearchCV): 0.83


c:\Users\Michael Brown\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Лучшие параметры для логистической регрессии (Hyperopt): {'C': 17.154826121340424}
Train F1-score для логистической регрессии (Hyperopt): 0.91
Test F1-score для логистической регрессии (Hyperopt): 0.78
Лучшие параметры для случайного леса (Optuna): {'n_estimators': 190, 'max_depth': 29, 'min_samples_split': 0.36578759832873753, 'min_samples_leaf': 0.11285330286727908}
Train F1-score для случайного леса (Optuna): 0.7
Test F1-score для случайного леса (Optuna): 0.71


###Выводы:
***

В данной работе мной были проделаны следующие действия:

1. Подготовка данных:
- Загружены данные из CSV-файла.
- Разделены на признаки (X) и целевую переменную (y).
- Выполнено разделение данных на обучающий и тестовый наборы.

2. Обучение моделей:
- Обучена логистическая регрессия и случайный лес.

3. Оценка моделей:
- Выполнена оценка моделей.
- Вычислен F1-score для каждой модели.
- Выполнена оценка производительности с использованием кросс-валидации.

4. Подбор гиперпараметров:
- Использован GridSearchCV для оптимизации гиперпараметров логистической регрессии.
- Использован RandomizedSearchCV для оптимизации гиперпараметров случайного леса.
- Использован Hyperopt для оптимизации гиперпараметров логистической регрессии.
- Использован Optuna для оптимизации гиперпараметров случайного леса.

5. F1-Score:

| Метод              | Train Score | Test Score |
|:------------------:|:-----------:|:----------:|
| GridSearchCV       | 0.85        | 0.79       |
| RandomizedSearchCV | 0.99        | 0.83       |
| Hyperopt           | 0.91        | 0.78       |
| Optuna             | 0.70        | 0.71       |


6. Выводы:
- Обе модели (логистическая регрессия и случайный лес) обучены и оценены.
- Применены различные методы для подбора гиперпараметров.
- Получены оптимальные параметры для каждой модели с использованием GridSearchCV, RandomizedSearchCV, Hyperopt и Optuna.
- Подобранные параметры могут быть использованы для дальнейшего улучшения производительности моделей на новых данных.